In [2]:
#import dependencies
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import jlc
import scipy
import cv2
from PIL import Image
from utils import (estimate_bias, estimate_bias_coefs,norm_quantile,
                   norm_translation,process_vol,soft_threshold,
                   get_disc_2d,inspect_fiber_vol,filter_vol,
                   get_high_density_mask,make_dist_time_plot,create_fiber_video,
                   create_mp4_from_frame_folder,generate_results,
                   plot_mean_growth_rate,FiberGraph,
                   generate_results_branching)
from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#define filenames based on tif vols in subfolder /data/
filenames = list(Path("./data/").glob("*.tif"))
filenames = [str(f) for f in filenames]
print(filenames)
assert len(filenames)>0, "filenamees is empty, make sure you are in a folder with a /data/ subfolder that contains the tif volumes"

['data\\YCsgA_Exp2_Pos1.tif', 'data\\YCsgA_Exp2_Pos2.tif', 'data\\YCsgA_Exp2_Pos3.tif', 'data\\YCsgA_Exp2_Pos4.tif', 'data\\YCsgA_Exp3_Pos1.tif', 'data\\YCsgA_Exp3_Pos2.tif', 'data\\YCsgA_Exp3_Pos3.tif', 'data\\YCsgA_Exp3_Pos4.tif']


In [ ]:
#Process and save volumes (Takes a long time)
BEST_TRANS = []
for filename in filenames:
    savename = filename.replace("data","data_processed3")
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    vol, best_translation = process_vol(filename,smaller=False,reset_origo_on_bad_frame=False)
    jlc.save_tifvol(vol,savename)
    BEST_TRANS.append(best_translation)

    savename = filename.replace("data","data_filtered3")
    os.makedirs(os.path.dirname(savename),exist_ok=True)
    vol = filter_vol(vol,only_median=False)
    jlc.save_tifvol(vol,savename)
    print("done with ",filename)


In [ ]:
#Generate all results in the results folder (results/[name_of_vol]/...). (takes a long time)
#volumes must be processed beforehand (previous cell)
for i in range(8):
    generate_results(i,filenames)
for i in range(len(filenames)):
    generate_results_branching(i,filenames)

In [ ]:
#save dist vs time file based on generated results
dist_vs_time_all = np.zeros((0,3))
exp_name = []
for i in range(len(filenames)):
    name = filenames[i].split("YCsgA_")[-1][:-4]
    loadname = "results/"+name+"/dist_vs_time.csv"
    dist_vs_time = np.loadtxt(loadname,delimiter=",")
    dist_vs_time_all = np.vstack((dist_vs_time_all,dist_vs_time))
    exp_name.extend([name]*len(dist_vs_time))
dist_vs_time_all = dist_vs_time_all.astype(object)
dist_vs_time_all = np.hstack((dist_vs_time_all,np.array(exp_name)[:,None]))
header="time,distance,fiber_number,exp_name"
fmt="%d,%f,%d,%s"
np.savetxt("results/dist_vs_time_all.csv",dist_vs_time_all,delimiter=",",header=header,fmt=fmt)

In [4]:
#do manual inspection of fiber volumes (on a single volume, without saving plots like in generate_results)
i = 0
vol_vis = jlc.load_tifvol(filenames[i].replace("data","data_processed3")).transpose(0,2,1)
vol = jlc.load_tifvol(filenames[i].replace("data","data_filtered3")).transpose(0,2,1)

In [5]:
#view the segmentation volume. Navigate frames with arrow keys. Use scroll wheel or drag-and-drop to change spatial view
jlc.inspect_vol(vol)
jlc.zoom()

In [6]:
#view the filtered volume
jlc.inspect_vol(vol_vis)
jlc.zoom()

In [7]:
#create a fiber graph for the volume and display the unfiltered fibers with the last frame
fiber_graph = FiberGraph(radius=3,threshold=0.2)
mf = vol.shape[0]
fiber_graph.process_vol(vol,max_frames=mf)
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot()
jlc.zoom()

In [8]:
#display in navigable form
inspect_fiber_vol(vol_vis,fiber_graph=fiber_graph,vmax=0.5)
jlc.zoom()

In [9]:
#filter the fibers, show the filtered fibers with the last frame
print(fiber_graph.num_fibers())
fiber_graph.cc_crop()
fiber_graph.leaf_crop()
fiber_graph.high_dens_crop(get_high_density_mask(vol[mf-1]))
fiber_graph.remove_inactive()
print(fiber_graph.num_fibers())
plt.figure()
plt.imshow(vol[mf-1],cmap="gray")
fiber_graph.plot(t=mf+1)
jlc.zoom()

525
352


In [10]:
#find high confidence paths and show the high cond paths
stats, growth_rate, paths = fiber_graph.estimate_stats(vol_vis[-1]-vol_vis[0].mean(),high_dens_image=get_high_density_mask(vol[-1]),max_dist=0,min_len=8)
inspect_fiber_vol(vol_vis,fiber_graph=fiber_graph,vmax=0.5,fiber_plot_kwargs={"plot_node_idx": sum(paths,[]),"mpl_colors": False, "color": "red"})
jlc.zoom()

found n=32 candidate paths with len>=8
found n=32 candidate paths with mean min dist>0


In [34]:
#save all the frame for a fiber video 
create_fiber_video(vol_vis,fiber_graph,plot_kwargs={"lw":0.5,"ms":2})
create_mp4_from_frame_folder("./video/",delete_frames=True)

In [11]:
#make a high cond dist covered versus time plot. 
#if add_start_time=True, then the fibers are aligned as 
#they grew in time, otherwise they all start at t=0.
make_dist_time_plot(paths,growth_rate,fiber_graph,add_start_time = True,plot=True,save_fig=False,save_csv=False,
                        save_fig_name="results/dist_vs_time.png",
                        save_csv_name="results/dist_vs_time.csv")
jlc.zoom()

In [12]:
#inspect the fiber volume with a specific high confidence path 
#(look at indices in the previous cell plot, define idx as one 
# of 0,1,2,3,... which you would like to see growing in the volume)
idx = 3
inspect_fiber_vol(vol_vis,fiber_graph=fiber_graph,vmax=0.5,fiber_plot_kwargs={"plot_node_idx": paths[idx],"mpl_colors": False, "color": "red"})
jlc.zoom()

In [20]:
#look at histogram of growth rates
time2 = []
dist2 = []
for dist,time in growth_rate:
    dist2 += dist.tolist()
    time2 += time.tolist()
rate = np.array(dist2)/np.array(time2)
#plt.scatter(time2,dist2)
plt.subplot(211)
plt.hist(rate,np.linspace(0,3,100),density=True)
plt.xlabel("growth rate (pixels/frame)")
plt.ylabel("probability")
plt.subplot(212)
plt.hist(rate,np.logspace(-3,0.5,100),density=False)
plt.xscale("log")
plt.xlabel("growth rate (pixels/frame)")
plt.ylabel("count")

Text(0, 0.5, 'count')

In [ ]:
#plot branches of the fiber graph
plot_kwargs={"circle_branches": True,"mpl_colors": False, "color": "red"}
plt.figure()
plt.imshow(vol_vis[-1],cmap="gray")
fiber_graph.plot(t=vol_vis.shape[0],**plot_kwargs)
jlc.zoom()

In [25]:
#Visualize a high density mask
frame_number = vol.shape[0]-1 #for e.g. last frame
mask = get_high_density_mask(vol[frame_number])
alpha = 0.5
im = vol[mf-1][None].repeat(3,axis=0)
im[0] = mask*alpha+im[0]*(1-mask*alpha)
im[1] = im[1]*(1-mask*alpha)
im[2] = im[2]*(1-mask*alpha)
im = im.transpose(1,2,0)
plt.imshow(im,cmap="gray")
jlc.zoom()

In [27]:
#make a plot of high density fraction versus time (requires generated_results to be run)
high_dens_per_t = []
names = []
plt.figure(figsize=(14,8))
for i in range(len(filenames)):
    name = filenames[i].split("YCsgA_")[-1][:-4]
    names.append(name)
    loadname = "results/"+name+"/high_dens_per_t.txt"
    high_dens_per_t.append(np.loadtxt(loadname))
    plt.plot(high_dens_per_t[-1],label=name)


plt.xlabel("time (frames)")
plt.ylabel("high density fraction")
plt.legend(names)
#plt.savefig("results/high_dens_per_t.png",dpi=300)

In [14]:
#plot number of branches versus time (requires generated_results_branching to be run)
divide_by_num_fibers = False #if you want to plot number of branches per fiber instead of just absolute number
save=False #save figure in results folder

num_branches_per_t_all = []
names = [f.split("YCsgA_")[-1][:-4] for f in filenames]
plt.figure(figsize=(14,8))
for name in names:
    loadname = "results/"+name+"/num_branches_per_t.txt"
    if divide_by_num_fibers:
        num_branches_per_t_all.append(np.loadtxt(loadname)/np.loadtxt("results/"+name+"/num_fibers_per_t.txt"))
    else:
        num_branches_per_t_all.append(np.loadtxt(loadname))
    plt.plot(num_branches_per_t_all[-1],label=name)
mean_first_4 = np.mean(np.array(num_branches_per_t_all[:4]),axis=0)
plt.plot(mean_first_4,label="Exp 2 mean",linewidth=3,color="red")
mean_last_4 = np.mean(np.array(num_branches_per_t_all[-4:]),axis=0)
plt.plot(mean_last_4,label="Exp 3 mean",linewidth=3,color="black")
plt.xlabel("time (frames)")
if divide_by_num_fibers:
    plt.ylabel("number of branches per fiber")
else:
    plt.ylabel("number of branches")
plt.legend()
if save:
    if divide_by_num_fibers:
        plt.savefig("results/num_branches_per_t_per_fiber.png",dpi=300)
    else:
        plt.savefig("results/num_branches_per_t.png",dpi=300)